In [ ]:
%matplotlib widget
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import pickle
import os
import sys
sys.path.insert(1, '../')
sys.path.insert(1, '../agent')
sys.path.insert(1, '../price_pred')

import stock_env
from QLAgent import QNAgent
from helper import *

### Set all parameters

In [ ]:
name_file_data = 'AAPL.csv'
env_name = 'StockEnv-v0'

num_episodes = 1 # training epoches
render_p = 10 # print frequency
init_balance = 1000 # initial fund agent have

len_obs = 70 # observation length, number of days the agent look back
len_window = 100 # num of times trained each time
interval = 1 # interval of validation
overlap = 20 # overlapping between two consecutive training data 
batch_size = 1000 
a = [i/10 for i in range(-4,5,1)]
print(tuple(a))
action_list=tuple(a)
seed = 40

### Construct environment and agents

In [ ]:
np.random.seed(seed)
tf.random.set_seed(seed)
# Read data you want to use for trading
train_data, test_data = get_data(f'../data/{name_file_data}')

In [ ]:
# Create an instant
env = gym.make(env_name, train_data=train_data, eval_data=test_data, 
               len_obs=len_obs, len_window=len_window, init_balance=init_balance,
               action_list=action_list)
env.seed(seed)
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

In [ ]:
# Create an agent
agent = QNAgent(env.action_space.n, env.observation_space.shape, 
                discount_rate=0.5, learning_rate=0.01, epsilon=0.01)

In [ ]:
init_ep = 0
train_statistics = pd.DataFrame()
test_statistics = pd.DataFrame()

### Train Agent

In [ ]:
worth = []
losses = []
for ep in range(init_ep, num_episodes):
    _, _, loss = get_performance(env, agent, train_data=True, training=True, batch_size=batch_size)
    if (ep % render_p) == 0:
        env.render(ep)
    worth.append(env.net_worth)
    losses = np.concatenate((losses,loss))
    if ep % interval == 0:
        overlap = overlap
        results_train = np.empty(shape=(0, 3))
        results_test = np.empty(shape=(0, 3))

        size_test = ((len(env.eval_data)-env.len_obs-env.len_window) // overlap)+1
        cagr_train, vol_train, _ = get_performance(env, agent, train_data=True, training=False, batch_size=size_test)
        results_train = np.array([np.tile(ep, size_test), cagr_train, vol_train]).transpose()

        cagr_test, vol_test, _ = get_performance(env, agent, train_data=False, training=False, overlap=overlap, batch_size=size_test)
        results_test = np.array([np.tile(ep, size_test), cagr_test, vol_test]).transpose()

        train_statistics = pd.concat([train_statistics, pd.DataFrame(results_train, columns=['epoch', 'cagr','volatility'])])
        test_statistics = pd.concat([test_statistics, pd.DataFrame(results_test, columns=['epoch', 'cagr','volatility'])])

### Statistics

#### Loss

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(losses)
plt.show()

#### Worth histogram

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(np.array(worth).flatten(), bins=range(500,2000,50))
plt.show()

#### Mean worth

In [ ]:
precision = num_episodes
threshold = 5000
worth_mean = np.mean(worth,axis=-1)
k = np.split(worth_mean, precision)
k = np.mean(k, axis = -1)
#k = [len(np.where(i > threshold)[0]) for i in worth]
plt.figure(figsize=(8,4))
plt.plot(k,'-')
#plt.boxplot(np.transpose(worth))

#### Save the model

In [ ]:
def save_ql_agent(agent, filename):
    os.mkdir(filename)
    param = (agent.action_size, agent.state_size, agent.epsilon, agent.discount_rate, agent.learning_rate, agent.optimizer)
    pickle.dump(param, open(filename + '/param', 'wb'))
    agent.model.save_weights(filename + '/model')

In [ ]:
save_ql_agent(agent, 'ql_model')

#### Load model

In [ ]:
def load_ql_agent(filename):
    # Obtain parameter
    param = pickle.load(open(filename + '/param', 'rb'))
    # Initialize Agent
    a = QNAgent(param[0], param[1])
    # Set parameter
    a.set_param(param)
    # Load model weight
    a.model.load_weights(filename+'/model')
    return a

In [ ]:
t = load_ql_agent('ql_model')